# HLLSet Swarm: State-to-State Mapping

Explore entanglement, trajectory (D,R,N), and state mappings.

## Architecture

```text
MOS(t-1) ←——Entanglement——→ MOS(t)
    ↓                          ↓
  W(t-1)  ——morphism——→    W(t)
    ↓                          ↓
  Basic HLLSets ——map——→  Basic HLLSets
    ↓                          ↓
  Trajectory: D (drift), R (retention), N (novelty)
```

Where:
- **D (Drift)**: Information lost from t-1 to t
- **R (Retention)**: Information preserved
- **N (Novelty)**: New information at t
- **Φ = N - D**: Noether current (should be ≈ 0)

In [1]:
from core.manifold_os import ManifoldOS
from core.hrt import HRTConfig
from core.state_mapping import TrajectoryTriple, StateMapping
from core.entanglement import EntanglementMorphism
import numpy as np

print('✓ Imports successful')

✓ Imports successful


## 1. Create MOS with HRT

In [2]:
# Use small config for fast computation
config = HRTConfig(p_bits=6, h_bits=8)
mos = ManifoldOS(hrt_config=config)

# Add perceptrons (sensors)
mos.add_perceptron('visual', 'camera')
mos.add_perceptron('audio', 'microphone')
mos.add_perceptron('tactile', 'touch_sensor')

print(f'MOS initialized:')
print(f'  Perceptrons: {len(mos.perceptrons)}')
print(f'  HRT dimension: {mos.hrt_factory.config.dimension}')
print(f'  Basic HLLSets: {mos.hrt_factory.config.num_basic_hllsets}')

MOS initialized:
  Perceptrons: 3
  HRT dimension: 514
  Basic HLLSets: 1028


## 2. Process Multiple States

In [3]:
# State t-2
print('Processing state t-2...')
mos.process_cycle({
    'visual': {'red', 'green'},
    'audio': {'low', 'mid'},
    'tactile': {'soft', 'rough'}
})
mos.commit('state_t-2')

# State t-1
print('Processing state t-1...')
mos.process_cycle({
    'visual': {'blue', 'yellow'},
    'audio': {'high', 'bass'},
    'tactile': {'smooth', 'warm'}
})
mos.commit('state_t-1')

# State t
print('Processing state t...')
mos.process_cycle({
    'visual': {'purple', 'orange'},
    'audio': {'treble', 'sub'},
    'tactile': {'cold', 'wet'}
})

print(f'\n✓ {len(mos.store.states)} states stored')
print(f'✓ {len(mos.store.hrts)} HRTs stored')

Processing state t-2...
Processing state t-1...
Processing state t...

✓ 2 states stored
✓ 3 HRTs stored


## 3. Map Current State to Previous

In [4]:
# Map MOS(t) → MOS(t-1)
mapping = mos.map_to_previous_state()

print('State Mapping Results:')
print(f'  Source (t-1): {mapping.source_state[:16]}...')
print(f'  Target (t):   {mapping.target_state[:16]}...')
print(f'\n  Entanglement:')
print(f'    Total pairs: {mapping.entanglement.total_pairs:,}')
print(f'    Max degree diff: {mapping.entanglement.max_degree_diff:.2f}')

KeyboardInterrupt: 

## 4. Trajectory Analysis (D, R, N)

In [5]:
trajectory = mapping.trajectory

print('Trajectory Triple (D, R, N):')
print(f'  ┌─────────────────────────────┐')
print(f'  │ Drift    (D): {trajectory.drift:8.2f}   │')
print(f'  │ Retention(R): {trajectory.retention:8.2f}   │')
print(f'  │ Novelty  (N): {trajectory.novelty:8.2f}   │')
print(f'  ├─────────────────────────────┤')
print(f'  │ Noether Φ:   {trajectory.noether_phi:8.4f}   │')
print(f'  │ Conserved:   {str(trajectory.is_conserved):>8}   │')
print(f'  │ Retention %: {trajectory.retention_ratio*100:7.1f}%   │')
print(f'  └─────────────────────────────┘')

# Interpretation
print('\nInterpretation:')
if trajectory.novelty > trajectory.drift:
    print('  → System is expanding (more novelty than drift)')
elif trajectory.drift > trajectory.novelty:
    print('  → System is contracting (more drift than novelty)')
else:
    print('  → System in balance')

if trajectory.is_conserved:
    print('  ✓ Noether conservation holds (|N| - |D| ≈ 0)')
else:
    print(f'  ⚠ Noether violation: Φ = {trajectory.noether_phi:.4f}')

NameError: name 'mapping' is not defined

## 5. State Decomposition

In [ ]:
print('Decomposition into Basic HLLSets:')
print(f'\n  Source (t-1):')
print(f'    Basic HLLSets used: {mapping.source_decomp.num_basic_used}')
print(f'    Coverage ratio: {mapping.source_decomp.coverage_ratio:.2%}')

print(f'\n  Target (t):')
print(f'    Basic HLLSets used: {mapping.target_decomp.num_basic_used}')
print(f'    Coverage ratio: {mapping.target_decomp.coverage_ratio:.2%}')

# Show some basic indices
source_indices = list(mapping.source_decomp.get_basic_indices())[:10]
print(f'\n  Source basic indices (first 10): {source_indices}')

## 6. Entanglement Morphism Details

In [ ]:
ent = mapping.entanglement

print('Entanglement Morphism W(t-1) → W(t):')
print(f'  Source lattice: {ent.source_lattice_hash[:16]}...')
print(f'  Target lattice: {ent.target_lattice_hash[:16]}...')
print(f'  Total entangled pairs: {ent.total_pairs:,}')

# Show sample mappings
print('\n  Sample mappings:')
shown = 0
for src_idx in list(ent.mapping.keys())[:5]:
    targets = ent.get_targets(src_idx)
    if targets:
        print(f'    Source {src_idx} → {len(targets)} targets')
        for tgt, strength in targets[:2]:
            print(f'      → Target {tgt} (s={strength:.3f})')
        shown += 1
if shown == 0:
    print('    (No strong entanglements found)')

## 7. Find Entangled Pairs

In [ ]:
# Find strongly entangled pairs
pairs = mos.find_entangled_pairs(threshold=0.0)

print(f'Entangled pairs (threshold=0.0): {len(pairs)}')
print('\nTop 10 strongest:')
for src, tgt, strength in pairs[:10]:
    print(f'  {src:4d} ↔ {tgt:4d}  strength={strength:.4f}')

# Statistics
if pairs:
    strengths = [s for _, _, s in pairs]
    print(f'\nStatistics:')
    print(f'  Mean strength: {np.mean(strengths):.4f}')
    print(f'  Max strength:  {np.max(strengths):.4f}')
    print(f'  Min strength:  {np.min(strengths):.4f}')

## 8. Map Any Two States

In [ ]:
# List available states
state_hashes = list(mos.store.states.keys())
print(f'Available states: {len(state_hashes)}')
for i, h in enumerate(state_hashes):
    print(f'  {i}: {h[:16]}...')

if len(state_hashes) >= 2:
    # Map first two states
    mapping2 = mos.map_states(
        state_hashes[0],  # oldest
        state_hashes[-1]  # newest
    )
    
    if mapping2:
        print(f'\nMapping oldest → newest:')
        print(f'  Trajectory: {mapping2.trajectory}')
        print(f'  Entanglement pairs: {mapping2.entanglement.total_pairs:,}')

## 9. Cumulative Trajectory

In [ ]:
# Map consecutive states and accumulate trajectories
trajectories = []

# Save current state
original_state = mos.current_state

for i in range(len(state_hashes) - 1):
    # Temporarily set current to target
    mos.current_state = mos.store.states[state_hashes[i+1]]
    
    # Map to previous
    m = mos.map_to_previous_state()
    if m:
        trajectories.append(m.trajectory)

# Restore
mos.current_state = original_state

# Accumulate
if trajectories:
    total_D = sum(t.drift for t in trajectories)
    total_R = sum(t.retention for t in trajectories)
    total_N = sum(t.novelty for t in trajectories)
    total_Phi = sum(t.noether_phi for t in trajectories)
    
    print('Cumulative Trajectory:')
    print(f'  Total Drift:    {total_D:.2f}')
    print(f'  Total Retention:{total_R:.2f}')
    print(f'  Total Novelty:  {total_N:.2f}')
    print(f'  Sum Noether Φ:  {total_Phi:.4f}')
    print(f'  Avg Noether Φ:  {total_Phi/len(trajectories):.4f}')

## 10. Statistics Summary

In [ ]:
stats = mos.stats()

print('='*50)
print('FINAL STATISTICS')
print('='*50)
print(f'Runtime:         {stats["runtime"]:.2f}s')
print(f'Processing cycles:{stats["processing_cycles"]}')
print(f'Perceptrons:     {stats["perceptrons"]}')
print(f'Persistent states:{stats["persistent_states"]}')
print(f'Persistent HRTs: {stats["persistent_hrts"]}')
print(f'HRT dimension:   {stats.get("hrt_dimension", "N/A")}')
print(f'Entanglement morphisms:{stats["entanglement_morphisms"]}')
print('='*50)

---

## Summary

This notebook demonstrated:

1. **HLLSet Swarm**: Collection of HLLSets at each time point
2. **State-to-State Mapping**: MOS(t-1) → MOS(t)
3. **Entanglement Morphism**: W(t-1) → W(t) via degree similarity
4. **Trajectory (D,R,N)**: Drift, Retention, Novelty analysis
5. **Noether Conservation**: |N| - |D| ≈ 0 check
6. **State Decomposition**: Into basic HLLSets

**Key Insight**: The system evolves through entangled mappings
between lattices, with trajectory tracking the flow of information.